In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from pyinn import conv2d_depthwise as conv2
from pyinn.modules import Conv2dDepthwise as Conv2dD
# import Transforms
%matplotlib inline
import pylab as pl
from IPython import display
import numpy as np
import time

In [2]:
data = torch.load("../Prototype_Network_Training/mIN_train.pth")

In [3]:
torch.cuda.set_device(3)
a = 256
b = 42
newdata = torch.stack([torch.ByteTensor(data[0][i]).float().view(84,84,3).transpose(0,2).contiguous() 
                       for i in range(64)],0)
newdata = Variable(newdata).cuda()
criterion = nn.MSELoss()
targ = Variable(torch.randn(64,b,b,a)).cuda()

In [4]:
from gpytorch.utils.fft import fftc,ifftc
from torch.autograd import Variable, Function
from torch.nn.parameter import Parameter

class FFT1(Function):
    def __init__(self):
        super(FFT1, self).__init__()
    
    def forward(self, inp):
        return fftc(inp)
    
    def backward(self, grad):
        return ifftc(grad)
    
class iFFT1(Function):
    def __init__(self):
        super(iFFT1, self).__init__()
        
    def forward(self, inp):
        return ifftc(inp)
    
    def backward(self, grad):
        return fftc(grad)
    
class Conv3dF(nn.Module):
    def __init__(self,nchannels):
        super(Conv3dF,self).__init__()
        self.nchannels = nchannels//2+1
        self.striper = Variable(torch.ones(1,nchannels,1,1), requires_grad=False).cuda()
                        #Conv2dD(channels = nchannels, kernel_size=3, padding=1)
#         self.mul = Parameter(torch.randn(1,nchannels,1,1))
        self.fft = FFT1()
        self.ifft = iFFT1()
        self.wre = Parameter(torch.randn(nchannels//2+1,1,3,3))
        self.wim = Parameter(torch.randn(nchannels//2+1,1,3,3))
#         self.convre = Conv2dD(channels = nchannels//2+1, kernel_size=3, padding=1)
#         self.convim = Conv2dD(channels = nchannels//2+1, kernel_size=3, padding=1)
                                                # TODO proper weight initialization
        
    def forward(self, inp):
        fi = self.fft(self.striper*inp)
        result = Variable(torch.zeros(*fi.size()), requires_grad=True).cuda()
        result[:,:,:,:,0].add(conv2(fi[:,:,:,:,0], self.wre, padding=1))
        result[:,:,:,:,0].sub(conv2(fi[:,:,:,:,1], self.wim, padding=1))
        result[:,:,:,:,1].add(conv2(fi[:,:,:,:,0], self.wim, padding=1))
        result[:,:,:,:,1].add(conv2(fi[:,:,:,:,1], self.wre, padding=1))
#         result.add(self.bias)
        
#         re = self.convre(fi[:,:,:,:,0])-self.convim(fi[:,:,:,:,1])
#         im = self.convim(fi[:,:,:,:,0])+self.convre(fi[:,:,:,:,1])
        
#         result = torch.cat([im.unsqueeze(4),
#                             re.unsqueeze(4)],
#                            dim=4)

#         k = fw[:,:,:,:,0]*fi.sum(4)
#         d = fi[:,:,:,:,1]*fw.sum(4)
#         c = fi[:,:,:,:,0]*(fw[:,:,:,:,0]-fw[:,:,:,:,1])
#         result = torch.cat([(k-d).unsqueeze(4),
#                             (k-c).unsqueeze(4)],dim=4)

        return self.ifft(result)

In [5]:
class testnet(nn.Module):
    def __init__(self):
        super(testnet, self).__init__()
        self.process = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Conv2d(3,a,(1,1),padding=0),
            nn.ReLU(),
            Conv3dF(a),
            nn.ReLU(),
            Conv3dF(a),
            nn.ReLU(),
            Conv3dF(a),
            nn.ReLU(),
            Conv3dF(a),
            nn.ReLU(),
            Conv3dF(a),
            nn.ReLU(),
            Conv3dF(a),
            nn.ReLU()
        )
        
    def forward(self, inp):
        return self.process(inp)

net = testnet().cuda()
nweights = sum([i.numel() for i in list(net.parameters())])
print(nweights," parameters in Fourier neural net.")

14956  parameters in Fourier neural net.


In [6]:
# FOURIER 3D
start = time.time()
for i in range(100):
    net.zero_grad()
    out = net(newdata)
    loss = criterion(out,targ)
    loss.backward()
    torch.cuda.synchronize()
print(time.time()-start)

33.32883977890015


In [7]:
#               84
# 1973 Mb  64       30.15
# 3401 Mb  128      59.38
# 6271 Mb  256      114.55
# DEAD     512
#               42
# 1067 Mb   64      8.69
# 1579 Mb  128      24.09
# 2641 Mb  256      50.69
# 4737 Mb  512      60.91   (!)
# 8913 Mb  1024     148.68  (???)
#               16
# 833 Mb   256 16   6.10
# 1137 Mb  512 16   11.42   (!)
# 1745 Mb  1024 16  27.67   (!)